# Model training
This notebook will guilde you to train a customised YOLO model with your own dataset (e.g. tank images)

---

### Environment setting up

_Go install cuda if you have nvdia GPU since it will accelerate your training process *a lot*_

Then you can go to [pyTorch official website](https://pytorch.org/get-started/locally/) to get the proper downloading command

In [3]:
# Example: Download pyTorch for windows with cuda
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 

# Install YOLO
!pip install ultralytics

# setting up the directory for YOLO training
import os

root_dir = 'tank_imgs'
if not os.path.exists(root_dir):
    os.makedirs(root_dir)

subdirs = ['train', 'valid']
for subdir in subdirs:
    subdir_path = os.path.join(root_dir, subdir)
    if not os.path.exists(subdir_path):
        os.makedirs(subdir_path)

for subdir in subdirs:
    for subsubdir in ['images', 'labels']:
        subsubdir_path = os.path.join(root_dir, subdir, subsubdir)
        if not os.path.exists(subsubdir_path):
            os.makedirs(subsubdir_path)

#### Data organisation for training
Before training your own customised YOLO model, put the:
* training images into `images` folder inside the `train` folder under `tank_imgs` directory
* labels for training images into `labels` folder at the same directory
* validation images into `images` folder inside the `valid` folder under `tank_imgs` directory
* labels for validation images into `labels` folder at the same directory

The training/validating ratio can be decided by yourself

In [ ]:
# training part
from ultralytics import YOLO

# the directory you want to put the yolo model in
dir = './'
# you can change the yolo versioin if you see fit
# we used nano model because it would later be deployed into Donkey Car's Raspberry Pi, which has limited calculation power
model = YOLO(f'{dir}/yolov8n.pt')

# start training
results = model.train(
    data = './weasel.yaml', # yaml file's directory and its name
    # imgsz=512, # input image size, default: 640
    epochs = 100,
    patience = 20, # awaiting n epochs, break if not improved
    batch = 16,
    project = 'weasel', # customised project name
    name = 'exp0' # customised name for this training
)

# After the long training done, you got your customised YOLO model!

---

### Converting the model into onnx format

ONNX is a light-weight format with compromised accuracy and fast inference speed
Since the calculation power of Raspberry Pi is limited, while real-time inference is neccessary for our WEASEL, onnx plays a crucial role therefore


In [ ]:
# from ultralytics import YOLO

model_name = 'yolov8m' #@param ["yolov8n", "yolov8s", "yolov8m", "yolov8l", "yolov8x"]
input_width = 640 #@param {type:"slider", min:32, max:4096, step:32}
input_height = 480 #@param {type:"slider", min:32, max:4096, step:32}
optimize_cpu = False

model = YOLO(f'{model_name}.pt') 
# If you don't have a Nvidia GPU, set `optimize_cpu = True` to optimize for CPU inference
model.export(format = 'onnx', imgsz = [input_height, input_width], optimize = optimize_cpu)

